**Overview**

This notebook explains how to run experiments using Sample-Factory, as well as upload and download models from the Hugging Face Hub. We will use OpenAI Gym's Lunar Lander environment as an example.

Note that this notebook is for demonstration purposes only. It is not recommended to run custom Sample-Factory experiments in Jupyter Notebooks due to Python Multiprocessisng not working with functions and objects not defined in an imported module. In this example notebook, we use environment functions defined in `sf_examples`to set up the gym environment and parse custom enviornments. Including the `make_gym_env_func` directly in this notebook will result in an error when running the experiment.


**Step 1: Install Dependencies**

To run this notebook, we need to install both `sample-factory` and the Lunar Lander environment using `pip`. Additional setup is required to use the Hugging Face Hub and instructions can be found at https://www.samplefactory.dev/get-started/huggingface/

In [ ]:
!pip install sample-factory
!pip install gym[box2d]

In [1]:
# Imports and setup
# General imports
import gym
from typing import Optional
from IPython.display import Video

# Imports from Sample Factory library
from sample_factory.envs.env_utils import register_env
from sample_factory.train import run_rl
from sample_factory.enjoy import enjoy
from sample_factory.huggingface.huggingface_utils import load_from_hf

# Imports specific for gym environment from sample factory examples
from sf_examples.train_gym_env import parse_custom_args, make_gym_env_func


**Step 2: Create Lunar Lander Environemnt and Specify Training Parameters**

First, we need to create the Lunar Lander training environment. We can do so using Sample-Factory's `register_env` to register the environment, along with a custom function to create an environment from its name. For gym environemnts, we can use `make_gym_env_func` from `sf_examples.train_gym_env`.

We also need to specify some parameters for our experiment. All experiments need to specify `algo` which is the algorithm used to train, `env` which is the environment we are running on, and `experiment` which is where to save the model after running the experiment.

Other training parameters can be specified as well. A full list of parameters can be found by running Sample-Factory with the `--help` flag.

See the `sf_examples` folder for examples of how to run experiments using Sample Factory in different environments. Some supported enviornments include Atari, DMLab, MuJoCo, and ViZDoom.

In [2]:
# Register Lunar Lander environment
register_env("LunarLanderContinuous-v2", make_gym_env_func)

# Initialize basic arguments for running the experiment. These parameters are required to run any experiment
# The parameters can also be specified in the command line
experiment_name = "lunar_lander_example"
argv = ["--algo=APPO", "--env=LunarLanderContinuous-v2", f"--experiment={experiment_name}"]

cfg = parse_custom_args(argv=argv, evaluation=False)

# The following parameters can be changed from the default
cfg.reward_scale = 0.05
cfg.train_for_env_steps = 1000000
cfg.gae_lambda = 0.99
cfg.num_workers = 20
cfg.num_envs_per_worker = 6
cfg.seed = 0

**Step 3: Run Experiment**

Next, we train the experiment using the parameters we specified above.

In [ ]:
run_rl(cfg)

**Step 4: Evaluating the Model and Uploading to the Hugging Face Hub**

After training the model, we can use the `enjoy` function to see how well it did. `enjoy` also allows us to generate a video with the `--save_video` flag, as well as upload the model to the Hub using the `--push_to_hub` flag. Make sure you also specify `--hf_repository` with your Hugging Face username and repository name in the form `<username>/<repo_name>`

In [ ]:
## Change this to your Hugging Face username
username = "andrewzhang505"

enjoy_args = ["--no_render", "--max_num_episodes=5", "--push_to_hub", f"--hf_repository={username}/{experiment_name}", "--save_video"]
cfg = parse_custom_args(argv=argv+enjoy_args, evaluation=True)
enjoy(cfg)

In [ ]:
Video(f"./train_dir/{experiment_name}/replay.mp4", embed=True)


**Step 5: Downloading Models from the Hub**

You can also download other models from the Hub for your own use as well. The following will download the model to `./train_dir/sf2-lunar-lander/` and you can use the model by specifying `--experiment=sf2-lunar-lander`

In [ ]:
load_from_hf("./train_dir", "andrewzhang505/sf2-lunar-lander")

download_args = ["--algo=APPO", "--env=LunarLanderContinuous-v2", "--experiment=sf2-lunar-lander", "--no_render", "--max_num_episodes=1", "--save_video"]
cfg = parse_custom_args(argv=download_args, evaluation=True)
enjoy(cfg)

Video(f"./train_dir/sf2-lunar-lander/replay.mp4", embed=True)

**Additional Resources**

For more information on using Sample-Factory, check out our website at https://www.samplefactory.dev/ and our github at https://github.com/alex-petrenko/sample-factory/